In [5]:
# Remove repo in local Colab storage if needed
!rm -r Differential-Equation-Solutions-with-Multilayer-Perceptron

In [6]:
# clone repo if we don't have in this session, specify branch after -b
!git clone -b main https://github.com/thesullivantage/Differential-Equation-Solutions-with-Multilayer-Perceptron
# if already exists, just pull in next cell

Cloning into 'Differential-Equation-Solutions-with-Multilayer-Perceptron'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 102 (delta 48), reused 77 (delta 31), pack-reused 0
Receiving objects: 100% (102/102), 23.19 KiB | 2.90 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [7]:
# Update (pull) repo from branch of our choosing
!cd Differential-Equation-Solutions-with-Multilayer-Perceptron && git checkout main && git pull

Already on 'main'
Your branch is up to date with 'origin/main'.
Already up to date.


In [8]:
import sys
import os
from importlib import reload

In [9]:
sys.path.insert(0, 'Differential-Equation-Solutions-with-Multilayer-Perceptron')

In [14]:
# to reload modules as we make changes
reload(loss)
reload(model)


<module 'model_pk.model' from '/content/Differential-Equation-Solutions-with-Multilayer-Perceptron/model_pk/model.py'>

In [15]:
import model_pk
from model_pk import loss, model

In [16]:
import sys
import tensorflow.keras as keras
import tensorflow as tf

In [17]:
# Range of independent variable values
start = 1 # Start value
limit = 100  # End value (exclusive)
delta = 0.1  # Step size

In [18]:
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
mlp = model.MLP(num_embed_layer=2, phys_dimension=1,embed_dim=32)
loss_fn = loss.LossODE(f0=0)


In [21]:
### TODO: do not standardize; only train on unit standardized interval

In [19]:
# Make simple ODE inputs for f'(x) = 2x
phys_dimension = 1
# Range of independent variable values
start = 0  # Start value
limit = 100  # End value (exclusive)
delta = 0.1  # Step size

# Create a TensorFlow array in the style of np.arange
inputs = tf.reshape(tf.range(start, limit, delta, dtype=tf.float32), (-1, phys_dimension))
# Calculate mean and standard deviation
mean, variance = tf.nn.moments(inputs, axes=[0])
std_dev = tf.sqrt(variance)

# Standardize the data
standardized_inputs = (inputs - mean) / std_dev

In [20]:
# Training loop
num_epochs = 5  # Define the number of epochs

### Trained using SGD optimization (MSE), without replacement
### n (dim. inputs) gradient updates
for epoch in range(1, num_epochs+1):
    ### increment squared loss over entire epoch (dataset)
    epoch_total_loss = 0
    for input_example in standardized_inputs:
        # input_example_reshaped = tf.reshape(input_example, (1, phys_dimension))input_example_reshaped = tf.reshape(input_example, (1,))
        input_example_reshaped = tf.reshape(input_example, (1,1))
        with tf.GradientTape() as tape:
            # Compute the loss for the current input example (squared error)
            loss_curr = loss_fn.compute_loss_element(mlp, input_example_reshaped)
            epoch_total_loss += loss_curr
        # Compute gradients and update model parameters
        gradients = tape.gradient(loss_curr, mlp.trainable_variables)
        optimizer.apply_gradients(zip(gradients, mlp.trainable_variables))
    epoch_avg_loss = epoch_total_loss / tf.cast(inputs.shape[0], tf.float32)
    print(f'Epoch {epoch} Average Loss: ', epoch_avg_loss)

Epoch 1 Average Loss:  tf.Tensor([[4.]], shape=(1, 1), dtype=float32)
Epoch 2 Average Loss:  tf.Tensor([[4.0000076]], shape=(1, 1), dtype=float32)
Epoch 3 Average Loss:  tf.Tensor([[4.000011]], shape=(1, 1), dtype=float32)
Epoch 4 Average Loss:  tf.Tensor([[4.0000076]], shape=(1, 1), dtype=float32)
Epoch 5 Average Loss:  tf.Tensor([[4.000007]], shape=(1, 1), dtype=float32)


In [ ]:
# Training loop
num_epochs = 25  # Define the number of epochs
for epoch in range(num_epochs):
    with tf.GradientTape() as tape:

        # Compute the loss
        loss_curr = loss_fn.custom_de_loss(inputs, mlp)

    # Compute gradients and update model weights
    gradients = tape.gradient(loss_curr, mlp.trainable_variables)
    optimizer.apply_gradients(zip(gradients, mlp.trainable_variables))

    # Print loss every epoch (or as needed)
    print(f"Epoch {epoch + 1}, Loss: {loss.numpy()}")

In [ ]:
import numpy as np

In [ ]:
np.linspace(-1, 1, 10)

array([-1.        , -0.77777778, -0.55555556, -0.33333333, -0.11111111,
        0.11111111,  0.33333333,  0.55555556,  0.77777778,  1.        ])